In [1]:
import numpy as np
from numba import njit
import h5py as hp

In [2]:
class likelihoodRatioInterval():
    ...

In [3]:
def getStepIndex(list_):
    # input an dict mapping n0 to mu_2(b). output its step's index
    list_diff = list_[1:] - list_[:-1]
    step_index_list = np.argwhere(list_diff>0) + 1
    return step_index_list

In [4]:
def findIndex(bkg, bkg_scan):
    idx = np.argmin(np.abs(bkg_scan - bkg))
    return idx

In [5]:
def findModifiedUpperLimit(n0, bkg, mu2_dic, step_index):
    # Step index is the index when mu2 is jumpping.
    bkg_scan = mu2_dic["bkg_scan"]
    mu2_list = mu2_dic[str(n0)]
    bkg_step_points = bkg_scan[step_index]
    mu2_step_points = mu2_list[step_index]
    index = findIndex(bkg, bkg_scan)
    for i in range(index.shape[0]-1):
        # find nearest bkg in the dictionary.
        if (bkg > bkg_step_points[i] & bkg < bkg_step_points[i+1]): # bkg in the abnormal part.
            # if mu is in the dip reigon, modified it!
            if mu2_list[index] <= mu2_step_points[i+1]:
                return mu2_step_points[i+1]
            else:
                return mu2_list[index]
        elif bkg < bkg_step_points[1]: # bkg in the left region.
            return mu2_list[index]
        elif bkg > last(bkg_step_points):
            println("Warning: Need to know the next step point!!! Hint: enlarge bkg_scan region")

In [6]:
def findUpperLimit(n0, bkg, mu2_dic):
    bkg_scan = mu2_dic["bkg_scan"]
    mu2_list = mu2_dic[str(n0)]
    index = findIndex(bkg, bkg_scan)
    return mu2_list[index]

In [7]:
def findLowerLimit(n0, bkg, mu1_dic):
    bkg_scan = mu1_dic["bkg_scan"]
    mu1_list = mu1_dic[str(n0)]
    index = findIndex(bkg, bkg_scan)
    return mu1_list[index]

In [8]:
def findInterval(n0, bkg):
    step_index = getStepIndex(mu2_dic["{}".format(n0)])
    if step_index.shape[0] == 0: # find whether it have pathology or not.
        mu2 = findUpperLimit(n0, bkg, mu2_dic)
        mu1 = findLowerLimit(n0, bkg, mu1_dic)
    else:
        mu2 = findModifiedUpperLimit(n0, bkg, mu2_dic, step_index)
        mu1 = findLowerLimit(n0, bkg, mu1_dic)
    return [mu1, mu2]

In [9]:
file = hp.File("likelihood_mu_bkg_data/mu_bkg_0_20.h5", "r")
print(file.keys())
bkg_scan = np.copy(file["bkg"])
mu2_dic = file["mu2"]
mu1_dic = file["mu1"]
# idx = getStepIndex(mu2_list)
# file.close()

<KeysViewHDF5 ['bkg', 'mu1', 'mu2']>


In [10]:
%timeit getStepIndex(bkg_scan)

85.4 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
findInterval(2, 3.0)

KeyError: "Unable to open object (object 'bkg_scan' doesn't exist)"

In [ ]:
if __name__ == "__main__":
    file = hp.File("likelihood_mu_bkg_data/mu_bkg_0_20.h5", "r")
    print(file.keys())
    bkg_scan = file["bkg"]
    mu2_list = file["mu2/0"]
    idx = getStepIndex(mu2_list)
    import matplotlib.pyplot as plt
    plt.plot(bkg_scan, mu2_list)
    plt.scatter(bkg_scan[idx], mu2_list[idx])
    plt.show()
    file.close()